# Assignment 6 <br/>


In [9]:
from dwave.system.samplers import DWaveSampler
from dimod.reference.samplers import ExactSolver
from dwave.cloud import Client
from dwave.system.composites import EmbeddingComposite

TestCPU = ExactSolver()

client = Client.from_config()
client.get_solvers()



[StructuredSolver(id='DW_2000Q_6'),
 BQMSolver(id='hybrid_binary_quadratic_model_version2'),
 DQMSolver(id='hybrid_discrete_quadratic_model_version1'),
 CQMSolver(id='hybrid_constrained_quadratic_model_version1'),
 StructuredSolver(id='Advantage2_prototype1.1'),
 StructuredSolver(id='Advantage_system4.1'),
 StructuredSolver(id='Advantage_system6.1')]

In [6]:
sampler_2000 = DWaveSampler(id='DW_2000Q_6')
sampler_Adv = DWaveSampler(id = 'Advantage_system4.1')

**Question 1** <br/>
Given the solution to every possible 3-SAT must have one of the parameters equal to binary 1 and the other parameters equal to 0 and that the QUBO generated to represent this 3-SAT assumes that two qubits must be the same in value and will together represent one parameter of the 3-SAT (q0 = q5). This defines the equation of the QUBO to be: <br/>
<div align = "center"> E(q) = (1/3)(q0 + q5) - (1/3)(q1 + q4) + (2/3)(q0q4 + q1q4 + q1q5) - (q0q5) <br/><br/> </div>
The QUBO representation is given by the following code (as was defined in class). For this code q0 = q30, q1 = q46, q4 = q31, and q5 = q45.

In [10]:
q_biases = {(30, 30): 0.33, (46, 46): -0.33, (31, 31): -0.33, (45, 45): 0.33}
c_strength = {(30, 31): 0.66, (46, 31): 0.66, (46, 45): 0.66, (30, 45): -1}
Q1 = dict(q_biases)
Q1.update(c_strength)

r = sampler_2000.sample_qubo(Q1, num_reads=1000)
for (sample, energy, num) in r.data():
    print(sample, "Energy: ", energy, "Occurrences: ", num)

{30: 1, 31: 0, 45: 1, 46: 0} Energy:  -0.33999999999999997 Occurrences:  428
{30: 0, 31: 0, 45: 0, 46: 1} Energy:  -0.33000000000000007 Occurrences:  261
{30: 0, 31: 1, 45: 0, 46: 0} Energy:  -0.32999999999999996 Occurrences:  303
{30: 1, 31: 0, 45: 1, 46: 1} Energy:  -0.009999999999999981 Occurrences:  1
{30: 1, 31: 1, 45: 1, 46: 0} Energy:  -0.009999999999999981 Occurrences:  1
{30: 1, 31: 0, 45: 0, 46: 1} Energy:  -2.7755575615628914e-17 Occurrences:  3
{30: 0, 31: 1, 45: 0, 46: 1} Energy:  -2.7755575615628914e-17 Occurrences:  1
{30: 0, 31: 0, 45: 0, 46: 0} Energy:  0.0 Occurrences:  2


By comparing these results to results gathered from a "brute force" solution, we can confirm that the QUBO does in fact compute the proper minimization solutions of the 3-SAT problem.

In [12]:
rCPU = TestCPU.sample_qubo(Q1)
for(sample, energy, num) in rCPU.data():
    print(sample, "Energy:", energy, "Number:", num)

{30: 1, 31: 0, 45: 1, 46: 0} Energy: -0.33999999999999997 Number: 1
{30: 0, 31: 0, 45: 0, 46: 1} Energy: -0.33 Number: 1
{30: 0, 31: 1, 45: 0, 46: 0} Energy: -0.33 Number: 1
{30: 1, 31: 1, 45: 1, 46: 0} Energy: -0.010000000000000009 Number: 1
{30: 1, 31: 0, 45: 1, 46: 1} Energy: -0.009999999999999898 Number: 1
{30: 0, 31: 0, 45: 0, 46: 0} Energy: 0.0 Number: 1
{30: 1, 31: 0, 45: 0, 46: 1} Energy: 0.0 Number: 1
{30: 0, 31: 1, 45: 0, 46: 1} Energy: 0.0 Number: 1
{30: 0, 31: 1, 45: 1, 46: 0} Energy: 0.0 Number: 1
{30: 1, 31: 0, 45: 0, 46: 0} Energy: 0.33 Number: 1
{30: 0, 31: 0, 45: 1, 46: 0} Energy: 0.33 Number: 1
{30: 1, 31: 1, 45: 0, 46: 0} Energy: 0.66 Number: 1
{30: 0, 31: 0, 45: 1, 46: 1} Energy: 0.66 Number: 1
{30: 1, 31: 1, 45: 1, 46: 1} Energy: 0.9800000000000001 Number: 1
{30: 1, 31: 1, 45: 0, 46: 1} Energy: 0.99 Number: 1
{30: 0, 31: 1, 45: 1, 46: 1} Energy: 0.99 Number: 1


These results outline that the lowest energy state occurs in one of three states (noting the qubit naming replacements listed above): (q0 = 1,q1 = 0, q4 = 0, q5 = 1), (q0 = 0, q1 = 1, q4 = 0, q5 = 0), and (q0 = 0, q1 = 0, q4 = 1, q5 = 0). These results line up with the results of the quantum computations done in the previous code section as the same states are identified as the minimums in the majority of the readings. Additionally, these results are consistent with 3-SAT problems as each of the solutions has one of the parameters set to 1 and the other two parameters set to 0 (noting that q0 and q5 are together set as one parameter).

**Question 2** <br/>
For this question, the 3-SAT can be encoded into the system using the EmbeddingComposite package. The following code encodes the given conditions.  

In [16]:
lin = {('z0', 'z0'): -1, ('z1', 'z1'): -2, ('z2', 'z2'): -3, ('z3', 'z3'): -3, ('z4', 'z4'): -3, ('z5', 'z5'): -3, ('z6', 'z6'): -3, ('z7', 'z7'): -3, ('z8', 'z8'): -2, ('z9', 'z9'): -1}

quad = {('z0', 'z1'): 2, ('z0', 'z2'): 2, ('z1', 'z2'): 4, ('z1', 'z3'): 2, ('z2', 'z3'): 4, ('z2', 'z4'): 2, ('z3', 'z4'): 4, ('z3', 'z5'): 2, ('z4', 'z5'): 4, ('z4', 'z6'): 2, ('z5', 'z6'): 4, ('z5', 'z7'): 2, ('z6', 'z7'): 4, ('z6', 'z8'): 2, ('z7', 'z8'): 4, ('z7', 'z9'): 2, ('z8', 'z9'): 2}
Q2 = dict(lin)
Q2.update(quad)

rCPU2 = TestCPU.sample_qubo(Q2)
#for(sample, energy, num) in rCPU2.data():
    #print(sample, "Energy:", energy, "Number:", num)



In [ ]:
def

**Question 3**

**Question 4**